In [26]:
from getpass import getpass
from importlib import reload
import src.drug_discovery_agent.agent as agent
from langchain_openai import ChatOpenAI
import os
from pathlib import Path
reload(agent)

<module 'src.drug_discovery_agent.agent' from '/home/dcutting/Projects/llm-agent-drug-discovery/src/drug_discovery_agent/agent.py'>

In [3]:
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")

In [27]:
vectordb_dir = Path("vector_db_dir/")
paper_path=Path("../data/example_drug_discovery_paper.pdf")
vectordb_dir.mkdir(exist_ok=True)


In [28]:
my_drug_discovery_chatbot = agent.DrugDiscoveryAgent(
    paper_path=paper_path,
    retriever_llm=ChatOpenAI(name='gpt-4-turbo'),
    vectordb_dir = vectordb_dir,
    verbose_agent = True,
    retriever_search_k=5
)

In [29]:
prompt_1 = "What disease are they discussing in the paper?"
prompt_2 = "Summarize the main takeaways from the paper."
prompt_3 =  "Can you list the protein targets they highlight in the paper?"
prompt_4 =  "Can you list 10 small molecule drugs they highlight in the paper?"
prompt_5 = "Lookup the smile strings associated with the small molecule drugs found in the paper. Write them alongside the drug names as a markdown table"
prompts = [prompt_1,prompt_2,prompt_3,prompt_4, prompt_5]
results = [my_drug_discovery_chatbot.chat(prompt, session_id=2) for prompt in prompts]

Parent run ac2d0783-7701-43d6-9713-151c6985fc85 not found for run 2ba457d9-04ed-4003-b9e8-775c16bf32ab. Treating as a root run.




> Entering new AgentExecutor chain...

Invoking: `query_pdf` with `{'query': 'What disease are they discussing in the paper?'}`


Non-Small Cell Lung Cancer (NSCLC)

2024-05-06 15:11:31.040 | INFO     | src.drug_discovery_agent.agent:chat:301 - Total Tokens: 1433
2024-05-06 15:11:31.041 | INFO     | src.drug_discovery_agent.agent:chat:302 - Prompt Tokens: 1422
2024-05-06 15:11:31.041 | INFO     | src.drug_discovery_agent.agent:chat:303 - Completion Tokens: 11
2024-05-06 15:11:31.042 | INFO     | src.drug_discovery_agent.agent:chat:304 - Successful Requests: 1
2024-05-06 15:11:31.043 | INFO     | src.drug_discovery_agent.agent:chat:305 - Total Cost (USD): 0.002155
Parent run a4d53980-579d-4554-adbb-1d42ce8ea6f8 not found for run 439e8b3f-6e4e-4f7b-ad5d-c0d357c5d634. Treating as a root run.


The disease discussed in the paper is Non-Small Cell Lung Cancer (NSCLC).

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `query_pdf` with `{'query': 'Summarize the main takeaways from the paper.'}`


The paper discusses predictive biomarkers in advanced NSCLC, including various gene mutations and fusions. It also highlights FDA-approved targeted agents and resistance mechanisms in EGFR-mutated NSCLC. Osimertinib is recommended as the standard of care for untreated EGFR mutant advanced NSCLC.

2024-05-06 15:11:37.039 | INFO     | src.drug_discovery_agent.agent:chat:301 - Total Tokens: 4688
2024-05-06 15:11:37.040 | INFO     | src.drug_discovery_agent.agent:chat:302 - Prompt Tokens: 4626
2024-05-06 15:11:37.041 | INFO     | src.drug_discovery_agent.agent:chat:303 - Completion Tokens: 62
2024-05-06 15:11:37.041 | INFO     | src.drug_discovery_agent.agent:chat:304 - Successful Requests: 1
2024-05-06 15:11:37.042 | INFO     | src.drug_discovery_agent.agent:chat:305 - Total Cost (USD): 0.007063000000000001
Parent run aff6cd3b-aacf-40b1-9257-34914c028aa8 not found for run 6bf24bb4-1796-438f-8e6b-cab12a87b0c4. Treating as a root run.


...

The protein targets highlighted in the paper are:
- Anaplastic lymphoma kinase (ALK) fusion oncogene
- ROS proto-oncogene 1 receptor tyrosine kinase (ROS1) gene fusions
- Sensitizing endothelial growth factor receptor (EGFR) gene mutations
- BRAF V600E point mutations
- Neurotrophin tyrosine kinase (NTRK) gene fusions
- c-MET exon 14 (METex14) skipping mutations
- RET rearrangements

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `drug_smiles_lookup` with `{'drug_names': ['Crizotinib', 'Alectinib', 'Ceritinib', 'Entrectinib', 'Osimertinib', 'Dabrafenib', 'Trametinib', 'Lorlatinib', 'Selpercatinib', 'Capmatinib']}`


[{'molecule_chembl_id': 'CHEMBL601719', 'canonical_smiles': 'C[C@@H](Oc1cc(-c2cnn(C3CCNCC3)c2)cnc1N)c1c(Cl)ccc(F)c1Cl', 'pref_name': 'CRIZOTINIB', 'synonym_name': 'Crizotinib'}, {'molecule_chembl_id': 'CHEMBL1738797', 'canonical_smiles': 'CCc1cc2c(cc1N1CCC(N3CCOCC3)CC1)C(C)(C)c1[nH]c3cc(C#N)ccc3c1C2=O', 'pref_name': 'ALECTINIB', 'synonym_name': 'Alecti

2024-05-06 15:11:50.749 | INFO     | src.drug_discovery_agent.agent:chat:301 - Total Tokens: 0
2024-05-06 15:11:50.750 | INFO     | src.drug_discovery_agent.agent:chat:302 - Prompt Tokens: 0
2024-05-06 15:11:50.751 | INFO     | src.drug_discovery_agent.agent:chat:303 - Completion Tokens: 0
2024-05-06 15:11:50.752 | INFO     | src.drug_discovery_agent.agent:chat:304 - Successful Requests: 0
2024-05-06 15:11:50.752 | INFO     | src.drug_discovery_agent.agent:chat:305 - Total Cost (USD): 0.0
Parent run 07d4de0c-2a69-4775-8a8e-3e63d79bbeb4 not found for run 6e3d2a12-7fa0-4cf1-8905-90c83831ea54. Treating as a root run.


The 10 small molecule drugs highlighted in the paper are:

1. Crizotinib
2. Alectinib
3. Ceritinib
4. Entrectinib
5. Osimertinib
6. Dabrafenib
7. Trametinib
8. Lorlatinib
9. Selpercatinib
10. Capmatinib

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `drug_smiles_lookup` with `{'drug_names': ['Crizotinib', 'Alectinib', 'Ceritinib', 'Entrectinib', 'Osimertinib', 'Dabrafenib', 'Trametinib', 'Lorlatinib', 'Selpercatinib', 'Capmatinib']}`


[{'molecule_chembl_id': 'CHEMBL601719', 'canonical_smiles': 'C[C@@H](Oc1cc(-c2cnn(C3CCNCC3)c2)cnc1N)c1c(Cl)ccc(F)c1Cl', 'pref_name': 'CRIZOTINIB', 'synonym_name': 'Crizotinib'}, {'molecule_chembl_id': 'CHEMBL1738797', 'canonical_smiles': 'CCc1cc2c(cc1N1CCC(N3CCOCC3)CC1)C(C)(C)c1[nH]c3cc(C#N)ccc3c1C2=O', 'pref_name': 'ALECTINIB', 'synonym_name': 'Alectinib'}, {'molecule_chembl_id': 'CHEMBL2403108', 'canonical_smiles': 'Cc1cc(Nc2ncc(Cl)c(Nc3ccccc3S(=O)(=O)C(C)C)n2)c(OC(C)C)cc1C1CCNCC1', 'pref_name': 'CERITINIB', 'synonym_name': 'Ceri

2024-05-06 15:12:08.549 | INFO     | src.drug_discovery_agent.agent:chat:301 - Total Tokens: 0
2024-05-06 15:12:08.550 | INFO     | src.drug_discovery_agent.agent:chat:302 - Prompt Tokens: 0
2024-05-06 15:12:08.551 | INFO     | src.drug_discovery_agent.agent:chat:303 - Completion Tokens: 0
2024-05-06 15:12:08.551 | INFO     | src.drug_discovery_agent.agent:chat:304 - Successful Requests: 0
2024-05-06 15:12:08.552 | INFO     | src.drug_discovery_agent.agent:chat:305 - Total Cost (USD): 0.0


Here are the SMILE strings associated with the small molecule drugs found in the paper:

| Drug Name     | SMILE String                                                                                      |
|---------------|---------------------------------------------------------------------------------------------------|
| Crizotinib    | C[C@@H](Oc1cc(-c2cnn(C3CCNCC3)c2)cnc1N)c1c(Cl)ccc(F)c1Cl                                           |
| Alectinib     | CCc1cc2c(cc1N1CCC(N3CCOCC3)CC1)C(C)(C)c1[nH]c3cc(C#N)ccc3c1C2=O                                    |
| Ceritinib     | Cc1cc(Nc2ncc(Cl)c(Nc3ccccc3S(=O)(=O)C(C)C)n2)c(OC(C)C)cc1C1CCNCC1                                 |
| Entrectinib   | CN1CCN(c2ccc(C(=O)Nc3n[nH]c4ccc(Cc5cc(F)cc(F)c5)cc34)c(NC3CCOCC3)c2)CC1                          |
| Osimertinib   | C=CC(=O)Nc1cc(Nc2nccc(-c3cn(C)c4ccccc34)n2)c(OC)cc1N(C)CCN(C)C                                    |
| Dabrafenib    | CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2F)c(-c2ccnc(N)n2)s1 

In [30]:
results

[{'input': 'What disease are they discussing in the paper?',
  'chat_history': [],
  'output': 'The disease discussed in the paper is Non-Small Cell Lung Cancer (NSCLC).'},
 {'input': 'Summarize the main takeaways from the paper.',
  'chat_history': [HumanMessage(content='What disease are they discussing in the paper?'),
   AIMessage(content='The disease discussed in the paper is Non-Small Cell Lung Cancer (NSCLC).')],
  'output': 'The main takeaways from the paper are:\n\n1. Discussion of predictive biomarkers in advanced Non-Small Cell Lung Cancer (NSCLC), including gene mutations and fusions.\n2. Highlighting FDA-approved targeted agents and resistance mechanisms in EGFR-mutated NSCLC.\n3. Recommendation of Osimertinib as the standard of care for untreated EGFR mutant advanced NSCLC.'},
 {'input': 'Can you list the protein targets they highlight in the paper?',
  'chat_history': [HumanMessage(content='What disease are they discussing in the paper?'),
   AIMessage(content='The diseas

In [31]:
from IPython.display import Markdown
Markdown("\n\n".join([result['output'] for result in results]))

The disease discussed in the paper is Non-Small Cell Lung Cancer (NSCLC).

The main takeaways from the paper are:

1. Discussion of predictive biomarkers in advanced Non-Small Cell Lung Cancer (NSCLC), including gene mutations and fusions.
2. Highlighting FDA-approved targeted agents and resistance mechanisms in EGFR-mutated NSCLC.
3. Recommendation of Osimertinib as the standard of care for untreated EGFR mutant advanced NSCLC.

The protein targets highlighted in the paper are:
- Anaplastic lymphoma kinase (ALK) fusion oncogene
- ROS proto-oncogene 1 receptor tyrosine kinase (ROS1) gene fusions
- Sensitizing endothelial growth factor receptor (EGFR) gene mutations
- BRAF V600E point mutations
- Neurotrophin tyrosine kinase (NTRK) gene fusions
- c-MET exon 14 (METex14) skipping mutations
- RET rearrangements

The 10 small molecule drugs highlighted in the paper are:

1. Crizotinib
2. Alectinib
3. Ceritinib
4. Entrectinib
5. Osimertinib
6. Dabrafenib
7. Trametinib
8. Lorlatinib
9. Selpercatinib
10. Capmatinib

Here are the SMILE strings associated with the small molecule drugs found in the paper:

| Drug Name     | SMILE String                                                                                      |
|---------------|---------------------------------------------------------------------------------------------------|
| Crizotinib    | C[C@@H](Oc1cc(-c2cnn(C3CCNCC3)c2)cnc1N)c1c(Cl)ccc(F)c1Cl                                           |
| Alectinib     | CCc1cc2c(cc1N1CCC(N3CCOCC3)CC1)C(C)(C)c1[nH]c3cc(C#N)ccc3c1C2=O                                    |
| Ceritinib     | Cc1cc(Nc2ncc(Cl)c(Nc3ccccc3S(=O)(=O)C(C)C)n2)c(OC(C)C)cc1C1CCNCC1                                 |
| Entrectinib   | CN1CCN(c2ccc(C(=O)Nc3n[nH]c4ccc(Cc5cc(F)cc(F)c5)cc34)c(NC3CCOCC3)c2)CC1                          |
| Osimertinib   | C=CC(=O)Nc1cc(Nc2nccc(-c3cn(C)c4ccccc34)n2)c(OC)cc1N(C)CCN(C)C                                    |
| Dabrafenib    | CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2F)c(-c2ccnc(N)n2)s1                                  |
| Trametinib    | CC(=O)Nc1cccc(-n2c(=O)n(C3CC3)c(=O)c3c(Nc4ccc(I)cc4F)n(C)c(=O)c(C)c32)c1                         |
| Lorlatinib    | C[C@H]1Oc2cc(cnc2N)-c2c(nn(C)c2C#N)CN(C)C(=O)c2ccc(F)cc21                                          |
| Selpercatinib | COc1ccc(CN2C3CC2CN(c2ccc(-c4cc(OCC(C)(C)O)cn5ncc(C#N)c45)cn2)C3)cn1                               |
| Capmatinib    | CNC(=O)c1ccc(-c2cnc3ncc(Cc4ccc5ncccc5c4)n3n2)cc1F                                                  |